# **Sport Image Classification**

### Team Members :
- FASSIER Thimothé
- LE ROUX Thomas
- MONGKHOUN Vincent

In this project, we are going to build a sport-image classifier. The model will take as an input an image and will return as an output the sport that is represented in it.

During this project, we have used some fonctions that were introduced during lecture labs about CNN and Computer Vision of this course.

This notebook of the project is divided into 4 parts :
- **Data Exploration and Data Pre-processing**
- **Transfer Learning with CNN features extraction**
- **Transfer Learning with Fine Tuning**
- **Conclusion & Improvement**

# Importation of packages

In [ ]:
import tensorflow as tf
import torch
import keras
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, classification_report
import cv2

import os
import shutil

# Data Exploration and Pre-Processing

## Test-data creation

The original dataset doesn't provide a folder where test data are located. Consequently, before beginning the data exploration and pre-precossing, we need to allocate some images as test data for the testing phase of our trained model

In [ ]:
Kaggle = True

if Kaggle : 
    DATASET_DIR = '../input/sports-image-dataset/sports-image-dataset'
else :
    DATASET_DIR = './sports-image-dataset'

DATA_DIR = os.path.join(DATASET_DIR, 'data')
sports = os.listdir(DATA_DIR)

print('The sports that our model will classify are the following :')
print(sports)

Let create a test data subfolder to store the data that will be used for the testing phase. We randomly select the data from each classes 

In [ ]:
# Creation of the test subfolder and the dataset

n_validation = 100

test_folder = os.path.join(DATASET_DIR, 'test_data')
if not os.path.exists(test_folder):
    os.mkdir(test_folder)
    for class_name in sports:
        train_subfolder = os.path.join(DATA_DIR, class_name)
        test_subfolder = os.path.join(test_folder, class_name)
        print("Populating %s..." % test_subfolder)
        os.mkdir(test_subfolder)
        images_filenames = sorted(os.listdir(train_subfolder))
        for image_filename in images_filenames[-n_validation:]:
            shutil.move(os.path.join(train_subfolder, image_filename),
                        test_subfolder)
        print("Moved %d images" % len(os.listdir(test_subfolder)))

## Creation of the tensorflow-datasets

As we have a set of images from our dataset that are filed into class-specific folders, we will use the method <code>tf.keras.utils.image_dataset_from_directory</code> to automatically process the data directly from the train subfolder and also to generate similar labeled dataset objects. 

We divide the data into 80% for the train dataset and 20% for the validation dataset.

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='rgb')

In [ ]:
val_dataset = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='rgb')

We can check if everything is good by looking at the shape of one batch

In [ ]:
# Check how the images are arranged within a batch 
for image_batch, labels_batch in train_dataset:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

Everything seems to be good as a batch of size 32 and the images are correctly converted into (img_height, img_width, nb_channels) format.

## Data Exploration

In [ ]:
class_names = train_dataset.class_names
nb_sports = len(class_names)
print('We will classify the folliwing sports : ' + str(class_names))
print('There is a total of ' + str(nb_sports) + ' sports')

In order to quickly switch between the name of the sport and the corresponding label, we define the following dictionay :

In [ ]:
# Create a quick path between label and sport
INDEX_TO_CLASS = {k: v for k, v in enumerate(class_names)}
INDEX_TO_CLASS

We can now plot one image of each class to see what it's look like

In [ ]:
# Plot one image per class
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(22):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

Let's now see what the images from the same class looks like. We first define a function that plot the different images from the same class

In [ ]:
def plot_classes(random_sport):
    '''Plot images from the same class'''

    n_rows = 3
    n_cols = 4
    sport_folder = os.path.join(DATA_DIR, random_sport)
    list_img = os.listdir(sport_folder)
    
    for row in range(n_rows):
        for col in range(n_cols):
            index = n_cols * row + col
            plt.subplot(n_rows, n_cols, index+1)

            # Pick a random images to plot
            for i in range(8):
                x = random.randint(0, len(list_img) - 1)
                image_ = plt.imread(os.path.join(sport_folder, list_img[x]))
                # Display the image
                plt.imshow(image_, cmap='binary', interpolation='nearest')
                plt.title(random_sport)
                plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
random_sport = INDEX_TO_CLASS[random.randint(0, nb_sports)]
print(f'The selected sport to display various images is {random_sport.upper()}')

plot_classes(random_sport)

In [ ]:
random_sport = INDEX_TO_CLASS[random.randint(0, nb_sports)]
print(f'The selected sport to display various images is {random_sport.upper()}')

plot_classes(random_sport)

In [ ]:
random_sport = INDEX_TO_CLASS[random.randint(0, nb_sports)]
print(f'The selected sport to display various images is {random_sport.upper()}')

plot_classes(random_sport)

We remark that within the same class, images are quiet different in term of orientation, color, zoom range or even light levels. There are even images of the equipment only, such as for hockey, basketball, badminton or football. The model should be agnostic or robust to these variations.  The idea of using data augmention becomes pertinent, and it's all the more the case as the train dataset is quite small for this type of case (Image Classification).


**Remark :** the images are not in the same size here but there will have the same during the training (cf Creation of Tensorflow Dataset section)

Let's check the number of images per each classes

In [ ]:
effectifs = []
for sport in class_names:
    effectif = os.listdir(os.path.join(DATA_DIR, sport))
    effectifs.append(len(effectif))

plt.figure(figsize=(25, 10))
sns.barplot(x=class_names, y=effectifs, palette='viridis')
plt.title('Images per Sports',)
plt.ylabel('Number of images')
plt.xlabel('Sports Name')
plt.tight_layout()
plt.show()

The 2 sports that are the most represented in the dataset are firstly Badminton and secondly Football. On the contrary, the ones that are the less represented are Basketball, Kabaddi and Chess. 

## Data-Preprocessing

We first configure the performance of the dataset.

The <code>Dataset.cache</code> keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training our model.

The <code>Dataset.prefetch</code> overlaps data preprocessing and model execution while training.

In [ ]:
train_dataset = train_dataset.cache().prefetch(buffer_size=10)
val_dataset = val_dataset.cache().prefetch(buffer_size=10)

We can noww configure the different data augmentation process that we will use during the training. Regarding the different variation of images within the same class, we have decided to do : 
- Horizontal and Vertical <code>RandomFlip</code>
- <code>RandomRotation</code> 

In [ ]:
# Create the random data augmentation 
data_augmentation = tf.keras.Sequential(
    [tf.keras.layers.RandomFlip("horizontal"),
     tf.keras.layers.RandomRotation(0.1),
    ]
)

We can test if everithing works well for one image

In [ ]:
for images, labels in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0), training=True)
        plt.imshow(augmented_image[0].numpy().astype("int32"))
        plt.title(INDEX_TO_CLASS[int(labels[0])])
        plt.axis("off")
plt.show()

# Transfer Learning with CNN Feature Extraction

In this section, we will implement a Transfer Learning model. We will use a pretrained CNN in order to process feature extraction. The aim is to, from a previous network, extract meaningful features from new samples. We will finally add a new model on top of this pretrained model so that we can repurpose the feature maps learned previously for the dataset.

## The model

The CNN that we are using is the <code>ResNet50</code> model, which is a quite good model that present a good balance between performance and time of computation

For feature extraction, we specify <code>the include_top=False</code> argument, so that when we load the network, it doesn't include the classification layers at the top. 

In [ ]:
base_model = tf.keras.applications.resnet50.ResNet50(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)  

We want to prevent the weights in a given layer of the <code>based_model</code> from being updated during training, so we freeze it : 

In [ ]:
# Freeze the base_model
base_model.trainable = False

# Let's see at the base model architecture
base_model.summary()

Once we have freezed the original top layer of the <code>Xception</code>, we can add our own classifier on the top of the model so it will be ready to be train with our own data.

In order to be fed to the CNN feature extractor, images have to be rescale. Then, we add a <code>Rescaling</code> layer to scale input values (initially in the <code>[0, 255]</code> range) to the <code>[-1, 1]</code> range. We also add a <code>Dropout</code> for regularization. 

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

# Create new model on top
inputs = tf.keras.Input(shape=(224, 224, 3))

# Data augmentation
x = data_augmentation(inputs)

# Pre-processing the data in order to be fed to the model
x = preprocess_input(x)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# Regularization
x = keras.layers.Dropout(0.2)(x)

outputs = keras.layers.Dense(22, activation="softmax")(x)

model = keras.Model(inputs, outputs)

model.summary()

We can now proceed to the training. We have decided to choose the following parameters : 
- <code>SGD</code> optimizer with default value of learning rate
- <code>SparseCategoricalCrossentropy</code> loss as we didn't onehot encoded our labels and that we 22 labels and not 2 (otherwise it we be <code>CategoricalCrossentropy</code>)
- <code>SparseCategoricalAccuracy</code> metric 

In [ ]:
# Definition of the optimization parameters and of the metric

model.compile(
    optimizer=tf.keras.optimizers.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
# Training

initial_epochs = 20
history_tl = model.fit(train_dataset,
                       epochs=initial_epochs,
                       validation_data=val_dataset)

## Perfomance of the model

### Loss and Accuracy

We can now look at learning curves of the training and validation accuracy/loss

In [ ]:
acc = history_tl.history['sparse_categorical_accuracy']
val_acc = history_tl.history['val_sparse_categorical_accuracy']

loss = history_tl.history['loss']
val_loss = history_tl.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Prediction of new samples and analysis

In [ ]:
TEST_DATA = os.path.join(DATASET_DIR, 'test_data')

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DATA,
    image_size=(img_height, img_width),
    color_mode='rgb')

In [ ]:
predictions = []
true_labels = []

for X, y in test_dataset :
    image_batch, label_batch = test_dataset.as_numpy_iterator().next()
    pred = model.predict_on_batch(image_batch)
    pred = tf.nn.softmax(pred)
    true_labels.append(label_batch)
    predictions.append(np.argmax(pred.numpy(),axis=1))

In [ ]:
predictions = [item for sublist in predictions for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
predictions_sports = [INDEX_TO_CLASS[k] for k in predictions]
true_labels_sports = [INDEX_TO_CLASS[k] for k in true_labels]

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print("Test loss :", loss)
print("Test accuracy :", accuracy)

In [ ]:
cf_matrix = confusion_matrix(true_labels_sports, predictions_sports, )
plt.figure(figsize=(20, 10))
sns.heatmap(cf_matrix, annot=True, cmap='Blues', xticklabels=sorted(
    set(true_labels_sports)), yticklabels=sorted(set(true_labels_sports)))
plt.xlabel("True Labels")
plt.ylabel("Predicted Labels")
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
 
print(classification_report(true_labels_sports, predictions_sports, target_names=class_names))

### Heatmap Visualization 

Def of a haetmap
To visualize the heatmap, we will follow the tutorial that is provided in the following link :  https://pyimagesearch.com/2020/03/09/grad-cam-visualize-class-activation-maps-with-keras-tensorflow-and-deep-learning/. We adapt this code in order to match with our

# Fine Tuning

In order to increase the performance even further, we will train (or "fine-tune") the weights of the top layers of the <code>Xception</code> pre-trained model alongside the training of the classifier we added. The training process will force the weights to be tuned from generic feature maps to features associated specifically with the dataset.

In [ ]:
base_model.trainable = True
model.summary()

In the cell above, we have unfreeze the weight. Consequently, as we are now training a much larger model and that we want to readapt the pretrained weights, it is important to use a lower learning rate for the optimizer. Otherwise, our model could overfit very quickly.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9),  
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_ft = model.fit(train_dataset,
                        epochs=total_epochs,
                        validation_data=val_dataset,
                        initial_epoch=history_tl.epoch[-1])

RQ: if the validation loss is much higher than the training loss, so you may get some overfitting.

## Loss and Accuracy

In [ ]:
acc += history_ft.history['sparse_categorical_accuracy']
val_acc += history_ft.history['val_sparse_categorical_accuracy']

loss += history_ft.history['loss']
val_loss += history_ft.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Prediction of new samples and analysis

In [ ]:
TEST_DATA = os.path.join(DATASET_DIR, 'test_data')

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DATA,
    image_size=(img_height, img_width),
    color_mode='rgb')

In [ ]:
predictions = []
true_labels = []

for X, y in test_dataset :
    image_batch, label_batch = test_dataset.as_numpy_iterator().next()
    pred = model.predict_on_batch(image_batch)
    pred = tf.nn.softmax(pred)
    true_labels.append(label_batch)
    predictions.append(np.argmax(pred.numpy(),axis=1))

In [ ]:
predictions = [item for sublist in predictions for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
predictions_sports = [INDEX_TO_CLASS[k] for k in predictions]
true_labels_sports = [INDEX_TO_CLASS[k] for k in true_labels]

In [ ]:
cf_matrix = confusion_matrix(true_labels_sports, predictions_sports, )
plt.figure(figsize=(20, 10))
sns.heatmap(cf_matrix, annot=True, cmap='Blues', xticklabels=sorted(
    set(true_labels_sports)), yticklabels=sorted(set(true_labels_sports)))
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
 
print(classification_report(true_labels_sports, predictions_sports, target_names=class_names))

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print("Test loss :", loss)
print("Test accuracy :", accuracy)

## Most confident mistakes

## Heatmap Visualization 

In [ ]:
last_conv_layer_name = "conv5_block3_out"

image_ = plt.imread('../input/sports-image-dataset/sports-image-dataset/test_data/football/00000764.JPG')
plt.imshow(image_, cmap='binary', interpolation='nearest')
plt.axis('off')
plt.show()

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [ ]:
from tensorflow.keras.applications.resnet50 import decode_predictions

last_conv_layer_name = 'conv5_block3_out'
# Prepare image
img_array = preprocess_input(get_img_array('../input/sports-image-dataset/sports-image-dataset/test_data/football/00000758.jpg', size=(224, 224, 3)))

# Remove last layer's softmax
base_model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)
pred = tf.nn.softmax(preds)
i = np.argmax(pred[0])
label = INDEX_TO_CLASS[i]

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

# Conclusion & Improvement

- The pretrained model was already very good. Fine tuning does not really seem to help. It might be more interesting to introspect the quality of the labeling in the training set to check for images that are too ambiguous and should be removed from the training set.